## Etape 1 : recupération des données avec api yahoo

In [ ]:

import yfinance as yf

symboles = "MC.PA TTE.PA OR.PA RMS.PA SAN.PA AIR.PA SU.PA BNP.PA GLE.PA ACA.PA CS.PA DG.PA VIE.PA ENGI.PA ORA.PA CAP.PA STMPA.PA SAF.PA HO.PA MT.AS RNO.PA ML.PA PUB.PA BN.PA CA.PA KER.PA LR.PA SGO.PA AI.PA EN.PA URW.PA WLN.PA VIV.PA TEP.PA DSY.PA"

print("Téléchargement en cours")

data = yf.download(symboles, period="max", group_by='ticker', auto_adjust=False, progress=False, threads=True)

df_final = data.stack(level=0).reset_index()

df_final.rename(columns={'level_1': 'Ticker'}, inplace=True)

print(f"Il y a  {len(df_final)} lignes.")
print(f"Colonnes : {df_final.columns.tolist()}")

df_final.to_csv("data/cac40.csv", index=False)

Téléchargement en cours


C:\Users\Tomle\AppData\Local\Temp\ipykernel_2108\2362134665.py:13: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_final = data.stack(level=0).reset_index()


Il y a  229699 lignes.
Colonnes : ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']


In [ ]:

import yfinance as yf
import pandas as pd

symboles_macro = "BZ=F NG=F GC=F SI=F HG=F EURUSD=X EURJPY=X EURGBP=X ^TNX ^VIX ^NDX BTC-USD"

print(f"Téléchargement de {len(symboles_macro.split())} indicateurs MACRO ")

data_macro = yf.download(symboles_macro, period="max", group_by='ticker', auto_adjust=False, progress=False, threads=True)

df_macro = data_macro.stack(level=0).reset_index()

df_macro.rename(columns={'level_1': 'Ticker'}, inplace=True)

print(f"Macro téléchargée : {len(df_macro)} lignes.")
print("Indicateurs récupérés :")
print(df_macro['Ticker'].unique())

df_macro.to_csv("data/macro.csv", index=False)

Téléchargement de 12 indicateurs MACRO 


C:\Users\Tomle\AppData\Local\Temp\ipykernel_18012\1846992572.py:13: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_macro = data_macro.stack(level=0).reset_index()


Macro téléchargée : 87789 lignes.
Indicateurs récupérés :
['^TNX' '^NDX' '^VIX' 'EURGBP=X' 'GC=F' 'HG=F' 'NG=F' 'SI=F' 'EURJPY=X'
 'EURUSD=X' 'BZ=F' 'BTC-USD']


Afin de répondre aux exigences de volume du projet (> 200 000 observations) , nous avons automatisé la constitution de notre jeu de données via le langage Python.

Plutôt que de télécharger manuellement des dizaines de fichiers, nous avons développé un script utilisant la librairie yfinance, qui interagit avec l'API publique de Yahoo Finance .

Notre démarche s'est déroulée en trois points clés :

Ciblage : Nous avons défini un panier d'actifs représentatif de l'économie française, composé des principales capitalisations du CAC 40 (LVMH, TotalEnergies, BNP Paribas, etc.).

Extraction Massive : Le script a récupéré l'historique complet de cotation (Ouverture, Plus haut, Plus bas, Clôture, Volume) pour chaque entreprise.

Structuration : Les données, initialement fragmentées, ont été fusionnées et pivotées (technique de stacking) pour obtenir un fichier CSV unique et structuré.

Ce processus nous a permis de générer instantanément une base de données brute de plus de 200 000 lignes, garantissant ainsi la robustesse statistique nécessaire pour nos futures analyses.

## Etape 2 : nettoyage des données

In [85]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data/cac40.csv.csv")

In [104]:
df_macro = pd.read_csv("data/macro.csv")

In [ ]:
print(df.shape)
df_macro.shape

(217403, 10)


(87789, 8)

In [ ]:
print(df.dtypes)
df_macro.dtypes

Date              datetime64[ns]
Ticker                    object
Open                     float64
High                     float64
Low                      float64
Close                    float64
Adj Close                float64
Volume                   float64
Rentabilite              float64
Volatilite_30j           float64
dtype: object


Date          object
Ticker        object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
dtype: object

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
print(df.dtypes)
df_macro['Date'] = pd.to_datetime(df_macro['Date'])
print(df_macro.dtypes)

Date              datetime64[ns]
Ticker                    object
Open                     float64
High                     float64
Low                      float64
Close                    float64
Adj Close                float64
Volume                   float64
Rentabilite              float64
Volatilite_30j           float64
dtype: object
Date              datetime64[ns]
Ticker                    object
Open                     float64
High                     float64
Low                      float64
Close                    float64
Adj Close                float64
Volume                   float64
Rentabilite              float64
Volatilite_30j           float64
dtype: object


I/ Nous avons converti explicitement cette colonne au format datetime64 à l'aide de la fonction pd.to_datetime().

Intérêt pour le projet : Cette conversion est un prérequis technique indispensable pour trois raisons :

Filtrage Temporel : Elle nous permet d'appliquer des opérateurs mathématiques sur le temps (ex: sélectionner uniquement les dates > 2000-01-01), ce qui est impossible sur du texte brut.

Tri Chronologique : Elle garantit que les observations sont bien ordonnées du passé vers le futur, ce qui est crucial pour le calcul des rendements (variation par rapport à la veille).

Visualisation : Elle permet aux bibliothèques graphiques (comme celles utilisées dans Streamlit) de reconnaître l'axe des abscisses comme une échelle de temps continue et non comme des catégories distinctes.

II/ Nous avons appliqué un filtre pour exclure toutes les observations antérieures au 1er Janvier 2000.

Justification économique et technique : Cette réduction du périmètre historique repose sur trois arguments majeurs :

Homogénéité monétaire (L'effet Euro) : L'analyse de séries temporelles financières sur une longue période est biaisée par les changements de devises. En nous concentrant sur la période post-2000, nous travaillons sur une ère économique cohérente marquée par l'adoption de l'Euro (introduit sur les marchés financiers en 1999), évitant ainsi les artefacts liés aux taux de conversion Franc/Euro des années 80-90.

Disponibilité des données (Data Quality) : La composition du CAC 40 a fortement évolué. De nombreuses entreprises technologiques ou industrielles actuelles n'étaient pas cotées ou n'existaient pas sous leur forme actuelle dans les années 1980. Démarrer en 2000 permet de maximiser le nombre d'entreprises ayant un historique complet et de réduire le nombre de valeurs manquantes (NaN) en début de jeu de données.

Pertinence des cycles boursiers : La période 2000-2025 couvre les cycles économiques modernes les plus pertinents pour notre analyse : l'éclatement de la bulle Internet (2000), la crise des Subprimes (2008), la crise de la dette souveraine (2011) et le choc du COVID-19 (2020).

In [110]:
df = df[df['Date'] >= '2000-01-01']
df_macro = df[df['Date']>='2000-01-01']

In [116]:
df.tail()

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Rentabilite,Volatilite_30j
229694,2025-11-24,TTE.PA,55.799999,56.240002,55.320000,55.889999,55.889999,10538741.0,-0.142937,1.075232
229695,2025-11-24,URW.PA,89.639999,90.080002,88.620003,89.459999,89.459999,934646.0,0.067111,0.891307
229696,2025-11-24,VIE.PA,28.750000,29.200001,28.650000,28.730000,28.730000,3214079.0,0.807016,0.972030
229697,2025-11-24,VIV.PA,2.550000,2.584000,2.501000,2.526000,2.526000,2415732.0,-0.551179,2.819770
229698,2025-11-24,WLN.PA,1.620000,1.650000,1.589000,1.598500,1.598500,2691937.0,0.408292,5.241926


In [117]:
print(f"Il y a {df.isna().sum().sum()} valeurs manquantes dans les actions")
print(f"Il y a {df.duplicated().sum()} valeurs en double dans les actions")
print(f"Il y a {df.isna().sum().sum()} valeurs manquantes dans les macros")
print(f"Il y a {df.duplicated().sum()} dans les macros")

Il y a 0 valeurs manquantes dans les actions
Il y a 0 valeurs en double dans les actions
Il y a 0 valeurs manquantes dans les macros
Il y a 0 dans les macros


Etape 3- Création de variables

Dans le but d'enrichir notre analyse et de dépasser la simple observation des prix bruts, nous avons généré deux indicateurs financiers fondamentaux :
1. La Rentabilité Journalière :
Les cours de bourse bruts sont difficilement comparables entre eux (ex: une action LVMH vaut environ 700€ alors qu'une action Orange vaut environ 10€).En calculant la variation en pourcentage par rapport à la veille (Daily Return), nous créons une métrique standardisée. Cela nous permet de comparer directement la performance de différentes entreprises, quelle que soit leur valorisation initiale, et d'identifier les jours de forte croissance ou de décroissance.

2. La Volatilité sur 30 jours (Volatilite_30j)
Méthodologie : Écart-type (Standard Deviation) des rentabilités calculé sur une fenêtre glissante (rolling window) de 30 jours.

Justification : La rentabilité seule ne suffit pas à évaluer un investissement ; il est crucial de mesurer le risque associé. Cette variable nous permet de quantifier l'instabilité du titre. Une volatilité élevée indique des variations de prix brutales (incertitude forte), tandis qu'une volatilité faible signale une valeur stable. Le choix d'une fenêtre de 30 jours permet d'observer l'évolution du risque mois par mois, lissant les bruits journaliers tout en restant réactif aux changements de tendance de marché

In [ ]:
df["Rentabilite"] = df.groupby('Ticker')['Close'].pct_change()*100

In [ ]:
print(f"il y a {df.isna().sum().sum()} valeurs manquantes")

il y a 35 valeurs manquantes


In [ ]:
df['Volatilite_30j'] = df.groupby('Ticker')['Rentabilite'].transform(lambda x: x.rolling(window=30).std())

In [96]:
print(f"il y a {df.isna().sum().sum()} valeurs manquantes")

il y a 1085 valeurs manquantes


In [ ]:
df.dropna(inplace=True)

print(f"Nombre de valeurs manquantes restantes : {df.isna().sum().sum()}")


df.to_csv("data/cac40_final.csv", index=False)
df_macro.to_csv('data/macros_final.csv',index=False)
print(" Fichier sauvegardé ")


Nombre de valeurs manquantes restantes : 0
Nombre total de lignes finales : 217403
 Fichier sauvegardé 


In [122]:
df_cac = pd.read_csv('data/cac40_final.csv')
df_macros = pd.read_csv("data/macros_final.csv")

df_final = pd.merge(df_cac, df_macros, on='Date',how='left')

In [124]:
df_final.tail()

,Date,Ticker_x,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Rentabilite_x,Volatilite_30j_x,Ticker_y,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,Rentabilite_y,Volatilite_30j_y
7156628,2025-11-24,WLN.PA,1.62,1.65,1.589,1.5985,1.5985,2691937.0,0.408292,5.241926,TTE.PA,55.799999,56.240002,55.320000,55.889999,55.889999,10538741.0,-0.142937,1.075232
7156629,2025-11-24,WLN.PA,1.62,1.65,1.589,1.5985,1.5985,2691937.0,0.408292,5.241926,URW.PA,89.639999,90.080002,88.620003,89.459999,89.459999,934646.0,0.067111,0.891307
7156630,2025-11-24,WLN.PA,1.62,1.65,1.589,1.5985,1.5985,2691937.0,0.408292,5.241926,VIE.PA,28.750000,29.200001,28.650000,28.730000,28.730000,3214079.0,0.807016,0.972030
7156631,2025-11-24,WLN.PA,1.62,1.65,1.589,1.5985,1.5985,2691937.0,0.408292,5.241926,VIV.PA,2.550000,2.584000,2.501000,2.526000,2.526000,2415732.0,-0.551179,2.819770
7156632,2025-11-24,WLN.PA,1.62,1.65,1.589,1.5985,1.5985,2691937.0,0.408292,5.241926,WLN.PA,1.620000,1.650000,1.589000,1.598500,1.598500,2691937.0,0.408292,5.241926


In [125]:
df_final.to_csv("data/data_final.csv", index=False)